In [1]:
import ee
ee.Initialize()

import geemap
import pandas as pd
import geopandas as gpd
import numpy as np 

import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

pd.options.display.float_format = '{:.3f}'.format

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


In [2]:
#pip install gspread oauth2client

#### area

In [20]:
aoi = ee.FeatureCollection('projects/ee-cocoacmr/assets/admin/cmr_admbnda_adm1')

#### input maps

In [21]:
lc_image = ee.Image('projects/ee-cocoacmr/assets/outputs/CMR_LC_2020')
tnt_image = ee.Image('projects/ee-cocoacmr/assets/outputs/CMR_TNTMMU_2020')
ipcc_image = ee.Image('projects/ee-cocoacmr/assets/outputs/CMR_IPCC_2020')
risk_image = ee.Image('projects/ee-cocoacmr/assets/outputs/CMR_EUDR_risk')
frag_image = ee.Image('projects/ee-cocoacmr/assets/outputs/CMR_TNTMMU_frag_2020')

In [22]:
images = [lc_image, tnt_image, ipcc_image,risk_image,frag_image]

In [23]:
# resolution of the maps
scale = 10

In [24]:
# band name 
lc_band = 'classification'
tnt_band = 'FNF_2020'
ipcc_band = 'remapped'
risk_band = 'risk'
frag_band = 'fragmentation'

In [25]:
band_names = ['classification','FNF_2020','remapped','risk','fragmentation']

In [26]:
# list of class codes
lc_labels = [11,12,13,14,15,21,22,30,41,42,50,60]
tnt_labels = [0,1]
ipcc_labels = [10,20,30,40,50,60]
risk_labels =[0,1,2,3,4,5,6,7,8,9,10]
frag_labels = [0,1,2,3,4]

In [27]:
class_label_sets = [
    [11,12,13,14,15,21,22,30,41,42,50,60],
    [0,1],
    [10,20,30,40,50,60],
    [0,1,2,3,4,5,6,7,8,9,10],
    [0,1,2,3,4]
]

In [28]:
# Function to compute area per class
def compute_area(image, band, label, valid_classes, region, region_name):
    classified = image.select(band)
    pixel_area = ee.Image.pixelArea().clip(region.geometry())
    area_image = pixel_area.addBands(classified)

    stats = area_image.reduceRegion(
        reducer=ee.Reducer.sum().group(groupField=1, groupName='class'),
        geometry=region.geometry(),
        scale=30,
        maxPixels=1e12
    )

    groups = ee.List(stats.get('groups'))

    def format_group(group):
        group = ee.Dictionary(group)
        cls = ee.Number(group.get('class'))
        return ee.Algorithms.If(
            valid_classes.contains(cls),
            {
                'Label': label,
                'Region': region_name,
                'Band': band,
                'Class': cls,
                'Area (ha)': ee.Number(group.get('sum')).divide(10000)
            },
            None
        )

    return groups.map(format_group).removeAll([None])

# Collect all results
results = ee.List([])

regions = aoi.toList(aoi.size())
n_regions = regions.size().getInfo()

for i in range(n_regions):
    feature = ee.Feature(regions.get(i))
    region_dict = feature.toDictionary().getInfo()
    region_name = region_dict['ADM1_EN']


    for img, band, label, class_labels in zip(images, band_names, labels, class_label_sets):
        result = compute_area(img, band, label, ee.List(class_labels), feature, region_name)
        results = results.cat(result)

# Convert to DataFrame
df = pd.DataFrame(results.getInfo())

# Add band_class as column header
df['Band_Class'] = df['Band'] + '_' + df['Class'].astype(str)

,Area (ha),Band,Class,Label,Region,Band_Class
0,3631428.698,classification,11,classification,Adamawa,classification_11
1,589348.694,classification,12,classification,Adamawa,classification_12
2,129710.100,classification,13,classification,Adamawa,classification_13
3,373253.893,classification,15,classification,Adamawa,classification_15
4,6.742,classification,21,classification,Adamawa,classification_21
...,...,...,...,...,...,...
322,32567.565,risk,10,risk,West,risk_10
323,6086.311,fragmentation,1,fragmentation,West,fragmentation_1
324,619059.246,fragmentation,2,fragmentation,West,fragmentation_2
325,105195.942,fragmentation,3,fragmentation,West,fragmentation_3


In [ ]:
df

In [29]:
# Create a unique key per region
df['Region'] = df['Region'].astype(str)

# Create band_class column
df['Band_Class'] = df['Band'] + '_' + df['Class'].astype(str)

# Pivot table: one row per Region, one column per Band_Class
df_pivot = df.pivot_table(
    index='Region',
    columns='Band_Class',
    values='Area (ha)',
    aggfunc='sum',
    fill_value=0
).reset_index()

# Optional: sort columns for consistency
df_pivot = df_pivot.sort_index(axis=1)

In [30]:
df_pivot

Band_Class,FNF_2020_0,FNF_2020_1,Region,classification_11,classification_12,classification_13,classification_14,classification_15,classification_21,classification_22,...,risk_1,risk_10,risk_2,risk_3,risk_4,risk_5,risk_6,risk_7,risk_8,risk_9
0,1651833.345,4747626.847,Adamawa,3631428.698,589348.694,129710.100,0.000,373253.893,6.742,203933.003,...,1257116.750,435530.310,5780.248,60382.120,156602.425,38485.020,85958.480,2268.167,3500838.709,831526.640
1,1266593.307,5605575.596,Centre,4056261.566,130780.826,55214.631,0.000,1351978.497,24696.051,38199.556,...,821555.307,1195234.397,75211.788,328303.292,578244.661,278995.446,132983.132,16571.275,1653442.039,1774061.210
2,908168.546,10090974.401,East,8843651.965,225014.764,62749.608,0.000,947641.219,4.818,24981.058,...,754639.951,5932069.744,32282.827,205984.235,331538.647,148535.892,126071.060,17252.145,1589842.288,1827986.010
3,3412491.302,3578.666,Far-North,3249.485,139.325,83.934,0.000,72.390,946.724,1442318.589,...,3060265.359,282961.738,0.000,0.000,49.429,0.000,1762.187,0.176,15952.965,3.609
4,341549.429,1679451.791,Littoral,1266710.533,17247.594,5296.776,76200.571,305484.917,11668.851,46745.143,...,198290.585,448963.641,78700.441,83127.432,166606.009,32235.382,72187.791,6394.254,383154.423,518504.978
5,4312365.132,2303214.931,North,1892514.334,292724.602,40423.268,0.000,57637.851,0.000,1481521.365,...,3602868.975,755012.696,0.000,6494.109,27739.222,4251.619,63539.447,18.799,1820799.606,288872.201
6,767834.686,973922.526,North-West,576320.273,127382.787,51468.509,0.000,212657.843,12.350,147964.789,...,562367.997,123744.149,2793.617,31306.564,100316.882,29812.792,17056.921,373.772,744597.091,123309.167
7,149428.067,4567884.806,South,3965759.760,22916.352,8656.476,712.832,543275.646,41897.847,30334.601,...,57393.602,1739731.385,222495.743,161596.106,150845.302,147422.705,115755.469,10107.605,345114.883,1762785.554
8,250621.878,2225802.165,South-West,1761133.227,20100.955,7588.800,134354.077,298523.836,49950.838,16624.785,...,111810.368,902101.864,90583.057,43761.534,140712.303,43429.252,50902.973,1054.218,412126.025,659536.402
9,660962.845,734471.739,West,431822.187,98811.725,38082.949,0.000,160742.846,33.369,181528.530,...,539334.759,32567.565,1073.603,28634.823,89652.290,10819.886,21180.442,1582.576,590103.649,63609.715


In [31]:
df_pivot.to_csv('/home/sepal-user/cocoa-CMR/CMR_map_areas.csv',index=True)